## MongoDB & LangChain Configuration

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [2]:
import os

In [3]:
os.environ["LANGCHAIN_PROJECT"]= os.getenv('LANGCHAIN_PROJECT')
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [4]:
MONGODB_ATLAS_CLUSTER_URI = os.getenv('MONGODB_ATLAS_CLUSTER_URI')
DB_NAME = os.getenv('DB_NAME')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
ATLAS_VECTOR_SEARCH_INDEX_NAME = os.getenv('ATLAS_VECTOR_SEARCH_INDEX_NAME')

In [5]:
COLLECTION_NAME

'embeddings'

In [6]:
from pymongo import MongoClient

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)
MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')


## Training

In [12]:
import os
from langchain_community.document_loaders import PyPDFLoader
pdfs = os.listdir('./training')
documents = []
for pdf in pdfs:
    loader = PyPDFLoader(f'./training/{pdf}')
    pages = loader.load_and_split()
    documents.extend(pages)
    
len(documents)

196

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter  # This splitter splits the chunks from the end of the line

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200, add_start_index = True)
chunks = text_splitter.split_documents(documents)
print(len(chunks))

443


In [16]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch

# insert the documents in MongoDB Atlas with their embedding
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection=MONGODB_COLLECTION
)